In [1]:
import bs4
import requests
import lxml
import re
import pandas
import matplotlib.pyplot as plt

In [2]:
import datetime
import numpy
import xlrd

In [3]:
data = {
        "vértices":[],
        "ETTJ IPCA":[],
        "ETTJ PRE":[],
        "Inflação Implícita":[],
    }

In [4]:
def adidata(resultado):
    if len(resultado) == 4:
        data['vértices'].append(int(resultado[0].replace('.',"")))
        data['ETTJ IPCA'].append(float(resultado[1].replace(',','.')))
        data['ETTJ PRE'].append(float(resultado[2].replace(',','.')))
        data['Inflação Implícita'].append(float(resultado[3].replace(',','.')))
    elif len(resultado) == 2:
        data['vértices'].append(int(resultado[0].replace('.',"")))
        data['ETTJ IPCA'].append(float(resultado[1].replace(',',".")))
        data['ETTJ PRE'].append(None)
        data['Inflação Implícita'].append(None)

In [5]:
def criar_ettj():
    
    res = requests.get("https://www.anbima.com.br/informacoes/est-termo/CZ.asp")
    soup = bs4.BeautifulSoup(res.text,'lxml')

    for i, tr in enumerate(soup.select('table')[6].select('tr')):
        if i < 5:
            for i, td in enumerate(tr.select('td')):
                if i < 2:
                    if re.search(r'\b\d{2}(?![,.])\b', td.text):
                        resultado = re.search(r'\b\d{2}(?![,.])\b', td.text)
                        data['vértices'].append(int(resultado[0].replace(',',"")))
                        data['ETTJ IPCA'].append(None)
                        data['Inflação Implícita'].append(None)
                    elif re.search(r'\d+,\d+',td.text):
                        resultado = re.search(r'\d+,\d+',td.text)
                        data['ETTJ PRE'].append(float(resultado[0].replace(',',".")))
    
    for item in soup.select('table')[5].select('tr'):
        resultado = re.findall(r'(?<!/)\d\d\d|(?<!/)\d+,\d+|(?<!/)\d+.\d+', item.text)
        adidata(resultado)
    
    return pandas.DataFrame(data)

In [6]:
df = criar_ettj()
del data

c:\Users\felip\anaconda3\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [7]:
def chamar_feriados():
    dados_feriados = pandas.read_excel("feriados_nacionais.xls")
    for i, d in enumerate(dados_feriados['Data']):
        if issubclass(type(d),datetime.datetime):
         pass
        else:
            dados_feriados = dados_feriados.drop(index=i)
    z = numpy.array(dados_feriados['Data'])
    z = z.astype('datetime64[D]')
    dados_feriados['Data'] = z
    return dados_feriados

In [8]:
df_feriados = chamar_feriados()

In [9]:
lista_vértices = df['vértices'].to_list()

In [10]:
def calcular_feriados(data_1,data_2):
    holydays = df_feriados.loc[(df_feriados['Data'] > data_1) & (df_feriados['Data'] < data_2)]['Data']
    holydays = numpy.array(holydays)
    holydays = holydays.astype('datetime64[D]')
    return holydays

In [11]:
def calcular_du(data_1,data_2,holydays):
    dias_uteis = numpy.busday_count(data_1, data_2, '1111100', holydays)
    return dias_uteis

In [12]:
def taxa_interpolada_linear(dias_uteis):
    tx_1 = []
    tx_2 = []
    
    for i in range(len(lista_vértices)-1):
        if lista_vértices[i] <= dias_uteis <= lista_vértices[i+1]:
            par = (lista_vértices[i],lista_vértices[i+1])

    indice = df[df['vértices'] == par[0]].index.to_list()
    indice = indice[0]

    tx_1.append(df[df.index == indice]['vértices'].values[0])
    tx_1.append(df[df.index == indice]['ETTJ PRE'].values[0])
    tx_2.append(df[df.index == indice+1]['vértices'].values[0])
    tx_2.append(df[df.index == indice+1]['ETTJ PRE'].values[0])

    tx_linear = tx_1[1] + (dias_uteis -tx_1[0])*(tx_2[1]-tx_1[1])/(tx_2[0]-tx_1[0])
    tx_linear = tx_linear/100
    
    return tx_linear
    

In [17]:
def criar_fluxo(data_2, data_1, fluxo, feriado):
    d = data_2
    while d > data_1:
        fluxo['Data Nominal'].append(d)
        if numpy.is_busday(d, holidays=feriado):
            d = numpy.busday_offset(d,offsets=0,roll='forward', holidays=feriado)
        fluxo['Data Efetiva'].append(d) 

        d = d - pandas.offsets.DateOffset(months=6)
        d = d.to_datetime64()
        d = d.astype('datetime64[D]')
        

In [52]:
fluxos = {
        "Data Nominal": [],
        "Data Efetiva":[],
        "Dias úteis":[],
        "Fluxo %":[],
        "Fluxo VP":[],
        "Fluxo Monetário":[],
    }

data1 = datetime.datetime.today()
data2 = datetime.datetime(2027,7,1)
data1 = numpy.datetime64(data1, 'D')
data2 = numpy.datetime64(data2, 'D')

cupom = 0.06
valor_face = 3000

feriados = calcular_feriados(data1,data2)

criar_fluxo(data2, data1, fluxos, feriados)

for i, data in enumerate(fluxos['Data Efetiva']):
    if i == 0:
        du = calcular_du(data1, data, feriados)
        fluxos['Dias úteis'].append(du)
        taxa = taxa_interpolada_linear(du)

        f_p = 1 + cupom
        fluxos["Fluxo %"].append(f_p)

        f_p_vp = (f_p)/(1+taxa)**(abs(du)/252)
        fluxos["Fluxo VP"].append(f_p_vp) 

        f_m_vp = f_p_vp*valor_face
        fluxos["Fluxo Monetário"].append(f_m_vp)

    else:
        du = calcular_du(data1, data, feriados)
        fluxos['Dias úteis'].append(du)
        taxa = taxa_interpolada_linear(du)

        f_p = cupom
        fluxos["Fluxo %"].append(f_p)

        f_p_vp = (f_p)/(1+taxa)**(abs(du)/252)
        fluxos["Fluxo VP"].append(f_p_vp) 

        f_m_vp = f_p_vp*valor_face
        fluxos["Fluxo Monetário"].append(f_m_vp)

In [53]:
df_fluxos = pandas.DataFrame(fluxos)

In [54]:
df_fluxos['Data Nominal'] = df_fluxos['Data Nominal'].dt.strftime('%d/%m/%Y')
df_fluxos['Data Efetiva'] = df_fluxos['Data Efetiva'].dt.strftime('%d/%m/%Y')

In [55]:
df_fluxos.to_excel('fluxo1.xlsx')